## Inhalt des Skripts

Ziele des Skripts ist pro START Konzept, vorerst in erster Linie ISIS Konzept über ein Mapping von Konzept zu Datapoint aus dem DPM Metadaten zu extrahieren. Das war ein alter Versuch, wo das Mapping auf Basis eines Datenabzugs erstellt wurde. Mittlerweile greifen wir direkt auf ITS Daten in HUE zu

# Setup

In [1]:
import pandas as pd
import os
import openpyxl
import numpy as np
import chardet


## START Konzepte

In [2]:
#File von Dominik/Christina zur Initialbefüllung des Datenkatalogs
# Relativer Pfad zur Datei
excel_file_path = "Files/ISIS-Erhebungsstammdaten1.xlsx"  

# Spezifisches Sheet einlesen
df_konzepte = pd.read_excel(excel_file_path, sheet_name="Gemeldete Konzepte")  

# Zeige die ersten Zeilen der Tabelle
print(df_konzepte.columns)

df_konzepte = df_konzepte[['Code' , 'Kurzbezeichnung (englisch)','Erhebungsteile']]
df_konzepte.head()

Index(['Code', 'Smart Cube Konzept', 'Pflichtkonzept', 'Dimensionskombination',
       'Dimensionen', 'Konzepttyp', 'OBServ-Schlüsselgruppe',
       'SCS-Einschränkung', 'Anubis-Rechenregel', 'Aggregationstyp',
       'Kurzbezeichnung', 'Kurzbezeichnung (englisch)', 'Bezeichnung',
       'Bezeichnung (englisch)', 'Beschreibung', 'Gültig von', 'Gültig bis',
       'MDI relevant', 'MDI Modellierungstyp', 'Erhebungsteile'],
      dtype='object')


,Code,Kurzbezeichnung (englisch),Erhebungsteile
0,IS01_FMA,NaN,NaN
1,IS01_SRB,NaN,NaN
2,IS02_FMA,NaN,NaN
3,IS02_SRB,NaN,NaN
4,IS03_FMA,NaN,NaN


In [3]:
#check for duplicates
has_duplicates = df_konzepte['Code'].duplicated().any()

print(has_duplicates)

df_konzepte = df_konzepte.rename(columns={"Code": "code"})

print(df_konzepte.size)


False
496026


## Mapping START Konzepte zu DPM Datapoint

In [4]:
# von https://pjira.w.oenb.co.at/browse/OSIJ-5164
csv_file_path = "Files/_select_k_code_mo_module_code_m_gueltig_von_m_gueltig_bis_dph_da_202408261249.csv"  

df_mapping = pd.read_csv(csv_file_path)
print(df_mapping.size)
df_mapping.head()


7610838


,code,module_code,gueltig_von,gueltig_bis,datapoint,taxonomy_code
0,OSLCRX1000001,COREP_LCR_DA_IND,2020-04-30 00:00:00.000,2021-06-29 00:00:00.000,37969,COREP_2.9.1
1,ISLCRX7210485,COREP_LCR_DA_IND,2020-04-30 00:00:00.000,2021-06-29 00:00:00.000,424361,COREP_2.9.1
2,ISLCRX7430150,COREP_LCR_DA_IND,2020-04-30 00:00:00.000,2021-06-29 00:00:00.000,425464,COREP_2.9.1
3,ISLCRX7430150,COREP_LCR_DA_IND,2020-04-30 00:00:00.000,2021-06-29 00:00:00.000,425474,COREP_2.9.1
4,ISLCRX7430150,COREP_LCR_DA_IND,2020-04-30 00:00:00.000,2021-06-29 00:00:00.000,425484,COREP_2.9.1


In [5]:
#Annahme: nur gueltig_bis > now oder nan sind relevant. Muss kontrolliert werden, vorallem mit neuen Daten!! 
arr_g_bis = df_mapping["gueltig_bis"].unique()
arr_g_bis_closed = arr_g_bis[~np.isin(arr_g_bis, ['2024-12-30 00:00:00.000', '3099-12-31 00:00:00.000'])]
arr_g_bis_closed = arr_g_bis_closed[~pd.isnull(arr_g_bis_closed)] #nan entfernen

print(len(arr_g_bis_closed))



27


In [6]:
#Der Datensatz enthält nur mehr Zeilen mit offenen oder zukünftigen gueltig_bis
df_mapping_g = df_mapping[~df_mapping['gueltig_bis'].isin(arr_g_bis_closed)]
print(df_mapping_g.size)
print(df_mapping_g.dtypes)
df_mapping_g.head()

1663482
code             object
module_code      object
gueltig_von      object
gueltig_bis      object
datapoint         int64
taxonomy_code    object
dtype: object


,code,module_code,gueltig_von,gueltig_bis,datapoint,taxonomy_code
5297,ISMX7010300,COREP_ALM_IND,2016-09-30 00:00:00.000,NaN,138791,COREP_2015-B-1
5298,ISMX7010400,COREP_ALM_IND,2016-09-30 00:00:00.000,NaN,139008,COREP_2015-B-1
5299,ISMX7010100,COREP_ALM_IND,2016-09-30 00:00:00.000,NaN,139449,COREP_2015-B-1
5300,ISMX7010200,COREP_ALM_IND,2016-09-30 00:00:00.000,NaN,139232,COREP_2015-B-1
5301,ISMX7010300,COREP_ALM_IND,2016-09-30 00:00:00.000,NaN,138798,COREP_2015-B-1


## Datenanalyse bzw. Merge von Konzept zu Mapping

In [7]:
df_mapping_g["taxonomy_code"]
distinct_values_list = df_mapping_g["taxonomy_code"].unique().tolist()
print(distinct_values_list)


['COREP_2015-B-1', 'COREP_2016-A', 'COREP_2014-03', 'COREP_2014-07', 'COREP_2015-A', 'COREP', 'COREP_2016-B', 'COREP_2017-A', 'RES_2.8.1', 'COREP_2.9', 'FINREP_2.9-Ind', 'FINREP_2.9', 'RES_2.9', 'FP_3.2.1', 'FINREP_3.2.1', 'FP_2.10', 'res_2021', 'GSII_3.0.1', 'GSII_3.0', 'FINREP_3.2.1-DP', 'REM_2.10', 'REM_3.0.1', 'REM_3.2.2', 'COREP_3.2', 'IRRBB_3.4', 'COVID19_3.2.1', 'AE_3.2', 'MREL_TLAC_3.2.1', 'ResRep_2024', 'GSII_3.2', 'ESG_3.3', 'IRRBB_3.3.2', 'IPU_3.3']


In [8]:
#wir mergen über einen Left - join die Mapping daten zu den Konzepten hinzu. mit validate: one_to_many
merged_df = pd.merge(df_konzepte, df_mapping_g, on="code", how="left", validate="one_to_many")

print(merged_df.shape)
print(merged_df.dtypes)
merged_df.head(5)
#merged_df.to_csv("merged_df.csv", index=False)

(377034, 8)
code                           object
Kurzbezeichnung (englisch)     object
Erhebungsteile                 object
module_code                    object
gueltig_von                    object
gueltig_bis                    object
datapoint                     float64
taxonomy_code                  object
dtype: object


,code,Kurzbezeichnung (englisch),Erhebungsteile,module_code,gueltig_von,gueltig_bis,datapoint,taxonomy_code
0,IS01_FMA,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,IS01_SRB,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,IS02_FMA,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,IS02_SRB,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,IS03_FMA,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
#wir droppen alle Zeilen ohne datapoint, wir wandeln datapoint in int um, der wird über merge zu float
df_cleaned = merged_df.dropna(subset=['datapoint'])
df_cleaned['datapoint'] = df_cleaned['datapoint'].astype('int')
print(df_cleaned.dtypes)
print(df_cleaned.shape)
df_cleaned.head()

code                          object
Kurzbezeichnung (englisch)    object
Erhebungsteile                object
module_code                   object
gueltig_von                   object
gueltig_bis                   object
datapoint                      int64
taxonomy_code                 object
dtype: object
(271758, 8)


/tmp/ipykernel_279/1689183507.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['datapoint'] = df_cleaned['datapoint'].astype('int')


,code,Kurzbezeichnung (englisch),Erhebungsteile,module_code,gueltig_von,gueltig_bis,datapoint,taxonomy_code
41296,ISAX0000001,Reporting Level,"AK-SD, AU-SD",AE,2024-09-30 00:00:00.000,NaN,37969,AE_3.2
41297,ISAX1000000,NaN,"AK-SD, AU-SD",AE,2023-06-30 00:00:00.000,NaN,31870,AE_3.2
41304,ISAX1001001,NaN,"AK-F3200, AU-F3200",AE,2023-06-30 00:00:00.000,NaN,41729,AE_3.2
41305,ISAX1001010,NaN,"AK-F3200, AU-F3200",AE,2023-06-30 00:00:00.000,NaN,41735,AE_3.2
41306,ISAX1001015,NaN,"AK-F3200, AU-F3200",AE,2023-06-30 00:00:00.000,NaN,465284,AE_3.2


## Import DPM data

In [10]:
#K:\HST-IT-Systeme\TP_2\Umsetzung\XBRL\EBA\DPM_3.5.0_Release_09_07_2024\dpm_databse_3.5_dpm_1.0.zip 
csv_file_path = "Files/qDPM_DataPointCategorisations.csv"  

with open("Files/qDPM_DataPointCategorisations.csv", 'rb') as file:
    result = chardet.detect(file.read())
    print(result)



{'encoding': 'Windows-1252', 'confidence': 0.73, 'language': ''}


In [11]:
df_dpm = pd.read_csv(csv_file_path, delimiter=';', encoding='Windows-1252', header=None, names= ["DataPointVID", "DataPointID", "DimensionLabel", "MemberName"])
print(df_dpm.size)
df_dpm.head()
#df_dpm.size


5752312


,DataPointVID,DataPointID,DimensionLabel,MemberName
0,10002,10002,Type of allowance,All allowances
1,10002,10002,Accounting portfolio,Accounting portfolios for financial assets sub...
2,10002,10002,Metric,Amount of accumulated impairment [mi]
3,10002,10002,Base,Assets
4,10002,10002,Main category,"Equity instruments, debt securities, loans and..."


In [12]:
distinct_values_list_dpm = df_dpm["MemberName"].unique().tolist()
#print(distinct_values_list_dpm)
len(distinct_values_list_dpm)

4828

## Merge Konzept über DataPointVID

In [13]:
#rename DataPointVID zu datapoint für merge
df_dpm = df_dpm.rename(columns={"DataPointVID": "datapoint"})

In [14]:
merged_df_final = pd.merge(df_cleaned, df_dpm, on="datapoint", how="left")

In [15]:
print(merged_df_final.shape)
distinct_values_list_dpm2 = merged_df_final["MemberName"].unique().tolist()
print(len(distinct_values_list_dpm2))
distinct_values_list_dpm3 = merged_df_final["code"].unique().tolist()
print(len(distinct_values_list_dpm3))
merged_df_final.head(30)

(2260456, 11)
3193
60066


,code,Kurzbezeichnung (englisch),Erhebungsteile,module_code,gueltig_von,gueltig_bis,datapoint,taxonomy_code,DataPointID,DimensionLabel,MemberName
0,ISAX0000001,Reporting Level,"AK-SD, AU-SD",AE,2024-09-30 00:00:00.000,NaN,37969,AE_3.2,37969,Metric,Reporting level [ei:SC:SC1_1]
1,ISAX0000001,Reporting Level,"AK-SD, AU-SD",AE,2024-09-30 00:00:00.000,NaN,37969,AE_3.2,37969,Base,Memorandum items
2,ISAX1000000,NaN,"AK-SD, AU-SD",AE,2023-06-30 00:00:00.000,NaN,31870,AE_3.2,31870,Metric,Accounting standard [ei:AS:AS1]
3,ISAX1000000,NaN,"AK-SD, AU-SD",AE,2023-06-30 00:00:00.000,NaN,31870,AE_3.2,31870,Base,Memorandum items
4,ISAX1001001,NaN,"AK-F3200, AU-F3200",AE,2023-06-30 00:00:00.000,NaN,41729,AE_3.2,41729,Metric,Carrying amount [mi]
5,ISAX1001001,NaN,"AK-F3200, AU-F3200",AE,2023-06-30 00:00:00.000,NaN,41729,AE_3.2,41729,Base,Assets
6,ISAX1001001,NaN,"AK-F3200, AU-F3200",AE,2023-06-30 00:00:00.000,NaN,41729,AE_3.2,41729,Encumbrance,Encumbered
7,ISAX1001001,NaN,"AK-F3200, AU-F3200",AE,2023-06-30 00:00:00.000,NaN,41729,AE_3.2,41729,Main category,All assets
8,ISAX1001010,NaN,"AK-F3200, AU-F3200",AE,2023-06-30 00:00:00.000,NaN,41735,AE_3.2,41735,Metric,Carrying amount [mi]
9,ISAX1001010,NaN,"AK-F3200, AU-F3200",AE,2023-06-30 00:00:00.000,NaN,41735,AE_3.2,41735,Base,Assets


In [16]:
merged_df_final.to_csv('output.dsv', index=False)


## Merge mit Zell-Koordinaten

In [ ]:
#File von Dominik/Christina zur Initialbefüllung des Datenkatalogs
# Relativer Pfad zur Datei
excel_file_path = "Files/TableCell.xlsx"  

# Spezifisches Sheet einlesen
df_konzepte = pd.read_excel(excel_file_path, sheet_name="TableCell")  